# Ball Bearing Fault Analysis

This notebook demonstrates condition-based monitoring techniques for detecting and diagnosing faults in ball bearings using vibration data analysis.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.fft import fft, fftfreq
from shmtools.features import SpectralFeatures, EnvelopeAnalysis
from shmtools.cbm import BearingFaultDetector

## 1. Generate Simulated Bearing Vibration Data

In [ ]:
# Simulation parameters
fs = 10000  # Sampling frequency (Hz)
duration = 2  # Duration (seconds)
shaft_speed = 30  # Hz (1800 RPM)

# Bearing geometry parameters
n_balls = 9
ball_diameter = 7.94  # mm
pitch_diameter = 39.04  # mm
contact_angle = 0  # degrees

# Calculate characteristic frequencies
BPFO = n_balls * shaft_speed * 0.5 * (1 - (ball_diameter/pitch_diameter) * np.cos(np.radians(contact_angle)))
BPFI = n_balls * shaft_speed * 0.5 * (1 + (ball_diameter/pitch_diameter) * np.cos(np.radians(contact_angle)))
BSF = (pitch_diameter/ball_diameter) * shaft_speed * 0.5 * (1 - (ball_diameter/pitch_diameter)**2 * np.cos(np.radians(contact_angle))**2)
FTF = shaft_speed * 0.5 * (1 - (ball_diameter/pitch_diameter) * np.cos(np.radians(contact_angle)))

print(f"Characteristic Frequencies:")
print(f"BPFO (Outer race): {BPFO:.2f} Hz")
print(f"BPFI (Inner race): {BPFI:.2f} Hz")
print(f"BSF (Ball spin): {BSF:.2f} Hz")
print(f"FTF (Cage): {FTF:.2f} Hz")

In [ ]:
# Generate time vector
t = np.linspace(0, duration, int(fs * duration))

# Simulate healthy bearing signal
healthy_signal = np.random.randn(len(t)) * 0.1  # Background noise
healthy_signal += 0.5 * np.sin(2 * np.pi * shaft_speed * t)  # Shaft rotation

# Simulate outer race fault
fault_signal = healthy_signal.copy()
# Add periodic impacts at BPFO frequency
impact_period = int(fs / BPFO)
for i in range(0, len(t), impact_period):
    if i + 50 < len(t):
        # Create exponentially decaying impact
        impact = 2 * np.exp(-100 * t[:50]) * np.sin(2 * np.pi * 2000 * t[:50])
        fault_signal[i:i+50] += impact

print(f"Generated {len(t)} samples at {fs} Hz")

## 2. Time Domain Analysis

In [ ]:
# Calculate time domain features
def calculate_time_features(signal):
    features = {
        'RMS': np.sqrt(np.mean(signal**2)),
        'Peak': np.max(np.abs(signal)),
        'Crest Factor': np.max(np.abs(signal)) / np.sqrt(np.mean(signal**2)),
        'Kurtosis': np.mean((signal - np.mean(signal))**4) / np.var(signal)**2
    }
    return features

healthy_features = calculate_time_features(healthy_signal)
fault_features = calculate_time_features(fault_signal)

print("Time Domain Features:")
print("\nHealthy Bearing:")
for key, value in healthy_features.items():
    print(f"  {key}: {value:.4f}")
print("\nFaulty Bearing:")
for key, value in fault_features.items():
    print(f"  {key}: {value:.4f}")

In [ ]:
# Plot time domain signals
fig, axes = plt.subplots(2, 1, figsize=(12, 8))

# Healthy signal
axes[0].plot(t[:1000], healthy_signal[:1000], 'b', alpha=0.7)
axes[0].set_title('Healthy Bearing Vibration')
axes[0].set_xlabel('Time (s)')
axes[0].set_ylabel('Acceleration (g)')
axes[0].grid(True)

# Fault signal
axes[1].plot(t[:1000], fault_signal[:1000], 'r', alpha=0.7)
axes[1].set_title('Bearing with Outer Race Fault')
axes[1].set_xlabel('Time (s)')
axes[1].set_ylabel('Acceleration (g)')
axes[1].grid(True)

plt.tight_layout()
plt.show()

## 3. Frequency Domain Analysis

In [ ]:
# Compute FFT
def compute_spectrum(signal, fs):
    N = len(signal)
    yf = fft(signal)
    xf = fftfreq(N, 1/fs)[:N//2]
    spectrum = 2.0/N * np.abs(yf[:N//2])
    return xf, spectrum

freq_healthy, spec_healthy = compute_spectrum(healthy_signal, fs)
freq_fault, spec_fault = compute_spectrum(fault_signal, fs)

# Plot frequency spectra
fig, axes = plt.subplots(2, 1, figsize=(12, 8))

# Healthy spectrum
axes[0].plot(freq_healthy[:2000], spec_healthy[:2000], 'b', alpha=0.7)
axes[0].set_title('Frequency Spectrum - Healthy Bearing')
axes[0].set_xlabel('Frequency (Hz)')
axes[0].set_ylabel('Amplitude')
axes[0].grid(True)
axes[0].set_xlim([0, 500])

# Fault spectrum with characteristic frequencies marked
axes[1].plot(freq_fault[:2000], spec_fault[:2000], 'r', alpha=0.7)
axes[1].axvline(x=BPFO, color='g', linestyle='--', label=f'BPFO ({BPFO:.1f} Hz)')
axes[1].axvline(x=2*BPFO, color='g', linestyle=':', alpha=0.5, label=f'2×BPFO')
axes[1].set_title('Frequency Spectrum - Bearing with Outer Race Fault')
axes[1].set_xlabel('Frequency (Hz)')
axes[1].set_ylabel('Amplitude')
axes[1].legend()
axes[1].grid(True)
axes[1].set_xlim([0, 500])

plt.tight_layout()
plt.show()

## 4. Envelope Analysis

In [ ]:
# Perform envelope analysis
def envelope_analysis(signal, fs, fc=2000, bw=1000):
    # Band-pass filter around resonance frequency
    nyq = fs / 2
    low = (fc - bw/2) / nyq
    high = (fc + bw/2) / nyq
    b, a = signal.butter(4, [low, high], btype='band')
    filtered = signal.filtfilt(b, a, signal)
    
    # Compute envelope using Hilbert transform
    analytic_signal = signal.hilbert(filtered)
    envelope = np.abs(analytic_signal)
    
    return envelope

# Compute envelopes
envelope_healthy = envelope_analysis(healthy_signal, fs)
envelope_fault = envelope_analysis(fault_signal, fs)

# Compute envelope spectra
freq_env_healthy, spec_env_healthy = compute_spectrum(envelope_healthy, fs)
freq_env_fault, spec_env_fault = compute_spectrum(envelope_fault, fs)

In [ ]:
# Plot envelope spectra
fig, axes = plt.subplots(2, 1, figsize=(12, 8))

# Healthy envelope spectrum
axes[0].plot(freq_env_healthy[:1000], spec_env_healthy[:1000], 'b', alpha=0.7)
axes[0].set_title('Envelope Spectrum - Healthy Bearing')
axes[0].set_xlabel('Frequency (Hz)')
axes[0].set_ylabel('Amplitude')
axes[0].grid(True)
axes[0].set_xlim([0, 200])

# Fault envelope spectrum
axes[1].plot(freq_env_fault[:1000], spec_env_fault[:1000], 'r', alpha=0.7)
axes[1].axvline(x=BPFO, color='g', linestyle='--', linewidth=2, label=f'BPFO ({BPFO:.1f} Hz)')
axes[1].axvline(x=2*BPFO, color='g', linestyle=':', alpha=0.7, label=f'2×BPFO')
axes[1].axvline(x=3*BPFO, color='g', linestyle=':', alpha=0.5, label=f'3×BPFO')
axes[1].set_title('Envelope Spectrum - Bearing with Outer Race Fault')
axes[1].set_xlabel('Frequency (Hz)')
axes[1].set_ylabel('Amplitude')
axes[1].legend()
axes[1].grid(True)
axes[1].set_xlim([0, 200])

plt.tight_layout()
plt.show()

## 5. Fault Severity Assessment

In [ ]:
# Calculate fault indicators
def calculate_fault_indicators(envelope_spectrum, freq, char_freq, harmonics=3):
    indicators = {}
    
    for h in range(1, harmonics + 1):
        target_freq = h * char_freq
        # Find amplitude at characteristic frequency
        idx = np.argmin(np.abs(freq - target_freq))
        indicators[f'{h}×BPFO'] = envelope_spectrum[idx]
    
    # Calculate total energy at fault frequencies
    total_energy = sum(indicators.values())
    indicators['Total Energy'] = total_energy
    
    return indicators

healthy_indicators = calculate_fault_indicators(spec_env_healthy, freq_env_healthy, BPFO)
fault_indicators = calculate_fault_indicators(spec_env_fault, freq_env_fault, BPFO)

print("Fault Indicators:")
print("\nHealthy Bearing:")
for key, value in healthy_indicators.items():
    print(f"  {key}: {value:.6f}")
print("\nFaulty Bearing:")
for key, value in fault_indicators.items():
    print(f"  {key}: {value:.6f}")

# Calculate severity ratio
severity_ratio = fault_indicators['Total Energy'] / healthy_indicators['Total Energy']
print(f"\nFault Severity Ratio: {severity_ratio:.2f}")

if severity_ratio > 10:
    print("Status: SEVERE FAULT - Immediate maintenance required")
elif severity_ratio > 5:
    print("Status: MODERATE FAULT - Schedule maintenance soon")
elif severity_ratio > 2:
    print("Status: EARLY FAULT - Monitor closely")
else:
    print("Status: HEALTHY - Continue normal operation")

## 6. Trending Analysis

In [ ]:
# Simulate fault progression over time
days = np.arange(0, 30)
fault_progression = 1 + np.exp((days - 20) / 5)  # Exponential degradation
fault_progression[fault_progression < 1] = 1

# Add some noise
fault_progression += np.random.randn(len(days)) * 0.2

# Plot trending
fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(days, fault_progression, 'b-o', label='Fault Indicator')
ax.axhline(y=2, color='y', linestyle='--', label='Warning Threshold')
ax.axhline(y=5, color='orange', linestyle='--', label='Alert Threshold')
ax.axhline(y=10, color='r', linestyle='--', label='Critical Threshold')

ax.fill_between(days, 0, 2, alpha=0.2, color='green', label='Normal')
ax.fill_between(days, 2, 5, alpha=0.2, color='yellow')
ax.fill_between(days, 5, 10, alpha=0.2, color='orange')
ax.fill_between(days, 10, max(fault_progression) + 1, alpha=0.2, color='red')

ax.set_xlabel('Days')
ax.set_ylabel('Fault Severity Indicator')
ax.set_title('Bearing Fault Progression Trending')
ax.legend(loc='upper left')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Estimate remaining useful life
critical_day = np.where(fault_progression > 10)[0]
if len(critical_day) > 0:
    rul = critical_day[0]
    print(f"Estimated Remaining Useful Life: {rul} days")
else:
    print("Bearing is healthy - RUL not applicable")

## Conclusion

This notebook demonstrated comprehensive ball bearing fault analysis including:

1. **Time Domain Analysis**: RMS, Peak, Crest Factor, and Kurtosis indicators
2. **Frequency Domain Analysis**: FFT spectrum showing characteristic fault frequencies
3. **Envelope Analysis**: Demodulation technique to enhance fault signatures
4. **Fault Severity Assessment**: Quantitative evaluation of bearing condition
5. **Trending Analysis**: Monitoring fault progression and estimating remaining useful life

Key findings:
- Outer race faults manifest at BPFO and its harmonics
- Envelope analysis effectively isolates bearing fault signatures
- Time domain features increase with fault severity
- Trending enables predictive maintenance scheduling